In [19]:
import os
# Check the latest 3.x version of Spark at https://downloads.apache.org/spark/ and specify it as the Spark version.
# For instance:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 2s (169 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to pro

In [20]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [21]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [22]:
# 2. Create a temporary view of the DataFrame.

spark.sparkContext.addFile(url)

# Read the CSV file into a DataFrame
sparkDF = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

sparkDF.createOrReplaceTempView("home_sales")

# Verify that the view has been created by running a basic query
spark.sql("SELECT * FROM home_sales LIMIT 5").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [23]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
averagePrice = spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) from home_sales where bedrooms == 4 group by YEAR(date) ORDER BY YEAR(date) desc")
averagePrice.show()


+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|           301819.44|      2021|
|           298353.78|      2020|
|            300263.7|      2019|
+--------------------+----------+



In [24]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avgerage3BedRoom = spark.sql("SELECT ROUND(AVG(price),2), date_built from home_sales where bedrooms == 3 AND bathrooms == 3 group by date_built ORDER BY date_built desc")
avgerage3BedRoom.show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           292676.79|      2017|
|           290555.07|      2016|
|            288770.3|      2015|
|           290852.27|      2014|
|           295962.27|      2013|
|           293683.19|      2012|
|           291117.47|      2011|
|           292859.62|      2010|
+--------------------+----------+



In [25]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
averagePrice200SQFT = spark.sql("SELECT ROUND(AVG(price),2), date_built from home_sales where bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000 group by date_built ORDER BY date_built desc")
averagePrice200SQFT.show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           280317.58|      2017|
|            293965.1|      2016|
|           297609.97|      2015|
|           298264.72|      2014|
|           303676.79|      2013|
|           307539.97|      2012|
|           276553.81|      2011|
|           285010.22|      2010|
+--------------------+----------+



In [26]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

averagePrice = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
    """)
averagePrice.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.7087161540985107 seconds ---


In [27]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [28]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [29]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_sales group by view having ROUND(AVG(price),2) >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

--- 0.9844250679016113 seconds ---


In [30]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sparkDF.write.parquet('home_parquet', mode='overwrite')

In [31]:
# 11. Read the formatted parquet data.
parQuetDF = spark.read.parquet('home_parquet')

In [32]:
# 12. Create a temporary table for the parquet data.
parQuetDF.createOrReplaceTempView('temp_parquet')

In [33]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from temp_parquet group by view having ROUND(AVG(price),2) >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  81|          1053472.79|
|  76|          1058802.78|
|  91|          1137372.73|
|  93|          1026006.06|
|  52|           733780.26|
|  86|          1070444.25|
|  94|           1033536.2|
|  57|            734340.5|
|  54|           798684.82|
|  96|          1017815.92|
|  92|           970402.55|
|  64|           767036.67|
|  61|           746877.59|
|  88|          1031719.35|
|  72|           780914.67|
|  59|            791453.0|
+----+--------------------+
only showing top 20 rows

--- 0.37835049629211426 seconds ---


In [34]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [35]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False